<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BA%D0%B5%D0%B9%D1%81%D1%8B%20%D0%BF%D0%BE%20%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%D1%83%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85/%D0%BA%D1%83%D1%80%D1%81%20%D0%B2%D0%B0%D0%BB%D1%8E%D1%82%D1%8B.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [0]:
from IPython.display import HTML

# Youtube
display(HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/v-rIM2YvTfA?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>'))

Разберемся, каким образом забирать данные о курсе валют в реальном времени с [сайта](https://www.cbr-xml-daily.ru/daily_json.js).

In [0]:
#!pip3 install requests  

Импортируем модуль и отправим запрос к сервису с курсами валют с помощью метода [GET](https://ru.wikipedia.org/wiki/HTTP#GET):

In [0]:
import requests  
response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')  

Посмотрим, что получили в ответ:

In [0]:
response

<Response [200]>

Мы получили объект ответа, который содержит всю нужную нам информацию. По умолчанию на экран выводится HTTP-код ответа 200. Это означает, что запрос был корректным, и сервер отдал нам нужную информацию.

Код ответа в виде числа можно получить с помощью метода `status_code`:

In [0]:
response.status_code

200

Адрес, по которому мы обращались, возвращает результат в `json` формате. Эти данные уже лежат в атрибуте `text` в полученном ответе `response`.

In [0]:
response.text[:230]

'{\n    "Date": "2020-05-23T11:30:00+03:00",\n    "PreviousDate": "2020-05-22T11:30:00+03:00",\n    "PreviousURL": "\\/\\/www.cbr-xml-daily.ru\\/archive\\/2020\\/05\\/22\\/daily_json.js",\n    "Timestamp": "2020-05-23T17:00:00+03:00",\n    "Va'

Сейчас текст хранится просто в строковой переменной. Далее мы можем превратить эту строку в словарь. Сделать это можно с помощью JSON-парсера либо воспользовавшись методом `json`, который уже встроен в объект ответа `response`.

In [0]:
currencies = response.json()
currencies 

{'Date': '2020-05-23T11:30:00+03:00',
 'PreviousDate': '2020-05-22T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2020/05/22/daily_json.js',
 'Timestamp': '2020-05-23T17:00:00+03:00',
 'Valute': {'AMD': {'CharCode': 'AMD',
   'ID': 'R01060',
   'Name': 'Армянских драмов',
   'Nominal': 100,
   'NumCode': '051',
   'Previous': 14.6658,
   'Value': 14.9238},
  'AUD': {'CharCode': 'AUD',
   'ID': 'R01010',
   'Name': 'Австралийский доллар',
   'Nominal': 1,
   'NumCode': '036',
   'Previous': 46.5332,
   'Value': 46.8373},
  'AZN': {'CharCode': 'AZN',
   'ID': 'R01020A',
   'Name': 'Азербайджанский манат',
   'Nominal': 1,
   'NumCode': '944',
   'Previous': 41.8061,
   'Value': 42.3698},
  'BGN': {'CharCode': 'BGN',
   'ID': 'R01100',
   'Name': 'Болгарский лев',
   'Nominal': 1,
   'NumCode': '975',
   'Previous': 39.7801,
   'Value': 40.0872},
  'BRL': {'CharCode': 'BRL',
   'ID': 'R01115',
   'Name': 'Бразильский реал',
   'Nominal': 1,
   'NumCode': '986',
   'Previ

Теперь данные лежат в словаре и мы можем легко получать необходимые значения.

In [0]:
currencies['Valute']['USD']  

{'CharCode': 'USD',
 'ID': 'R01235',
 'Name': 'Доллар США',
 'Nominal': 1,
 'NumCode': '840',
 'Previous': 70.924,
 'Value': 71.8804}

Рассмотрим датасет с зарплатами в ИТ:  

In [72]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/it_new.csv')
df.head(10)

,id,Дата рождения,Вакансия,Технология,Локация,З/п в валюте найма,col,curenc
0,1,18.04.1994,Инженер-программист 2 категории,JS,Санкт-Петербург,"140 000,00 ₽",140000.0,₽
1,2,27.03.1993,Разработчик информационных систем,.NET,Москва,"130 000,00 ₽",130000.0,₽
2,3,24.11.1983,Java Developer,Java,Санкт-Петербург,"200 000,00 ₽",200000.0,₽
3,4,09.06.1989,Архитектор ПО,Java,Москва,"250 000,00 ₽",250000.0,₽
4,5,03.10.1978,QA Automation Engineer,QA,Санкт-Петербург,"172 413,00 ₽",172413.0,₽
5,6,21.08.1973,Big Data Developer,Big Data,Санкт-Петербург,"280 000,00 ₽",280000.0,₽
6,7,10.10.1991,JS Developer,JS,Москва,"129 000,00 ₽",129000.0,₽
7,8,16.08.1982,Разработчик мобильных приложений,Android,Москва,"185 000,00 ₽",185000.0,₽
8,9,16.05.1988,Инженер-программист 2 категории,.NET,Санкт-Петербург,"160 000,00 ₽",160000.0,₽
9,10,01.12.1990,Java Engineer,Java,Краков; Польша,"9 500,00 zł",9500.0,zł


### Задание

Определите заработную плату в ₽ по текущему курсу.

In [0]:
df['curenc'] = df['curenc'].str.replace('zł','PLN').str.replace('Kč','CZK').str.replace('$', 'USD').str.replace('lev','BGN').str.replace('Ft','HUF').str.replace('€','EUR')

In [0]:
def wage_to_rub(row):
  return round(row['col']*currencies['Valute'][row['curenc']]['Value'],2) if row['curenc'] != '₽' else row['col']

In [81]:
df['Value in RUB'] = df.apply(wage_to_rub, axis = 1)
df

,id,Дата рождения,Вакансия,Технология,Локация,З/п в валюте найма,col,curenc,Value in RUB
0,1,18.04.1994,Инженер-программист 2 категории,JS,Санкт-Петербург,"140 000,00 ₽",140000.00,₽,140000.00
1,2,27.03.1993,Разработчик информационных систем,.NET,Москва,"130 000,00 ₽",130000.00,₽,130000.00
2,3,24.11.1983,Java Developer,Java,Санкт-Петербург,"200 000,00 ₽",200000.00,₽,200000.00
3,4,09.06.1989,Архитектор ПО,Java,Москва,"250 000,00 ₽",250000.00,₽,250000.00
4,5,03.10.1978,QA Automation Engineer,QA,Санкт-Петербург,"172 413,00 ₽",172413.00,₽,172413.00
5,6,21.08.1973,Big Data Developer,Big Data,Санкт-Петербург,"280 000,00 ₽",280000.00,₽,280000.00
6,7,10.10.1991,JS Developer,JS,Москва,"129 000,00 ₽",129000.00,₽,129000.00
7,8,16.08.1982,Разработчик мобильных приложений,Android,Москва,"185 000,00 ₽",185000.00,₽,185000.00
8,9,16.05.1988,Инженер-программист 2 категории,.NET,Санкт-Петербург,"160 000,00 ₽",160000.00,₽,160000.00
9,10,01.12.1990,Java Engineer,Java,Краков; Польша,"9 500,00 zł",9500.00,PLN,164644.50


### Возможности pandas для парсинга сайтов

Рассмотрим пример парсинга сайта с использованием возможностей модуля pandas:

In [0]:
import pandas as pd

df = pd.read_html("https://www.cbr.ru/key-indicators/")
df[0]

,0,1,2
0,валюта,22.05.2020,23.05.2020
1,Доллар США USD,709240,718804
2,Евро EUR,777965,784431


In [0]:
df[1]

,0,1
0,рублей за грамм,23.05.2020
1,Золото Au,"4 003,70"
2,Серебро Ag,3929
3,Платина Pt,"1 915,82"
4,Палладий Pd,"4 575,79"


In [0]:
df[2]

,0,1,2,3
0,% годовых,1 день,2-7 дней,8-30 дней
1,MIACR за 21.05.2020,545,528,—
2,RUONIA за 21.05.2020,532,NaN,NaN
3,% годовых,1 день,1 неделя,1 месяц
4,MosPrime Rate на 22.05.2020,562,572,579
5,% годовых,1 неделя,2 недели,1 месяц
6,ROISfix на 22.05.2020,541,541,532
